<a href="https://colab.research.google.com/github/messias077/REP/blob/main/Problema_Generalizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **# Avaliando a generalização de algoritmos**

In [6]:
# Importes necessários para os códigos 

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import RepeatedKFold, GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine

In [7]:
# Preparação para facilitar a avaliação dos modelos

""" Classe simplória para auxiliar na hora de rodar os modelos """
class Modelo():
    def __init__(self, modelo, descricao):
        self.modelo = modelo
        self.descricao = descricao
        self.acuracias = []  # Guarda as acurácias de cada rodada do modelo
    
    # Só para ficar mais simples na hora de chamar esses métodos
    def fit(self, X, y):
        self.modelo.fit(X, y)
    
    def predict(self, X):
        return self.modelo.predict(X)

# Carrega os dados
wine = load_wine()
X, y = wine.data, wine.target

# Guarda os modelos que serão rodados
modelos = []

# Definição dos modelos

# Logistic Regression convencional
lr_conv = LogisticRegression(max_iter=10000)
modelos.append(Modelo(lr_conv, 'Logistic Regression - Convencional (Sem normalização)'))

# Logistic Regression aninhado
parametros_lr = {'max_iter': [1000, 3000, 5000]}
grid_search_lr = GridSearchCV(LogisticRegression(), parametros_lr, scoring=make_scorer(accuracy_score))
lr_anin = Pipeline([
    ("padronizacao", StandardScaler()),
    ("classificador", grid_search_lr)
])
modelos.append(Modelo(lr_anin, 'Logistic Regression - Aninhado (Normalizado)'))

# KNN convencional
knn_conv = KNeighborsClassifier()
modelos.append(Modelo(knn_conv, 'KNeighbors Classifier - Convencional (Sem normalização)'))

# KNN Aninhado
parametros_knn = {'n_neighbors': [3, 5, 7]}
grid_search_knn = GridSearchCV(KNeighborsClassifier(), parametros_knn, scoring=make_scorer(accuracy_score))
knn_anin = Pipeline([
    ("padronizacao", StandardScaler()),
    ("gsknn", grid_search_knn)
])
modelos.append(Modelo(knn_anin, 'KNeighbors Classifier - Aninhado (Normalizado)'))

In [8]:
# Rodando e avaliando os modelos

def rodar_modelos(modelos, X, y):
    """
        Roda as folds diversas vezes (dependendo dos parâmetros passados) para
        cada modelo e acumula as acurácias para tirar a média no final das
        execuções.
    """
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)

    for m in modelos:
        scores = cross_validate(m.modelo, X, y, scoring='accuracy', cv=rkf)
        m.acuracias = scores['test_score']


# Roda os modelos
rodar_modelos(modelos, X, y)

print("\n  --------------------------------------")
print("\n         **** Resultados ****")
print("\n  --------------------------------------")

for mod in modelos:
    print(f"\n * {mod.descricao}")
    print(f"   - Acurácia global..: {np.mean(mod.acuracias)}")
    print(f"   - Desvio padrão....: {np.std(mod.acuracias)}\n")



  --------------------------------------

         **** Resultados ****

  --------------------------------------

 * Logistic Regression - Convencional (Sem normalização)
   - Acurácia global..: 0.956190476190476
   - Desvio padrão....: 0.029284746402898666


 * Logistic Regression - Aninhado (Normalizado)
   - Acurácia global..: 0.9775555555555556
   - Desvio padrão....: 0.02304051226299876


 * KNeighbors Classifier - Convencional (Sem normalização)
   - Acurácia global..: 0.6921587301587302
   - Desvio padrão....: 0.05015462108947563


 * KNeighbors Classifier - Aninhado (Normalizado)
   - Acurácia global..: 0.9579365079365079
   - Desvio padrão....: 0.032199929734872304

